In [ ]:
# Install the latest version of ParlAI from source.
# This is important, otherwise using Blenderbot 2.0 will not work.
!pip uninstall -q parlai -y 
!pip install -q  --progress-bar off git+https://github.com/facebookresearch/ParlAI.git 

# Install fairseq. This is required for Blenderbot 2
!pip install -q fairseq

# Clone the server and install its requirements
!git clone https://github.com/JulesGM/ParlAI_SearchEngine.git
!pip install -q  --progress-bar off -r ParlAI_SearchEngine/requirements.txt

In [ ]:
import os
import multiprocess
import shlex
import subprocess
import threading
import time

# Using port 8080 doesn't work on Colab
HOST = "0.0.0.0:1111"
# Change the following as neede
PATH_TO_SEARCH_SERVER = "./ParlAI_SearchEngine/search_server.py"

assert os.path.exists(PATH_TO_SEARCH_SERVER), (
    f"Incorrect path {PATH_TO_SEARCH_SERVER}"
)

command = ["python", "-u", shlex.quote(PATH_TO_SEARCH_SERVER), 
           "serve", "--host", HOST]
command_str = " ".join(command)
p = subprocess.Popen(
    command, 
    stderr=subprocess.STDOUT,
    stdout=subprocess.PIPE,
)

# Wait a bit before the next cell to let a lot of the potential errors happen.
time.sleep(1)

In [ ]:
# Test If the server crashed. 
# You can rerun this cell to check if the server crashed. 
if p.poll() is not None: 
    print(p.communicate()[0].decode())
    # If this says that the adress is already used, then it is very likely that
    # you already started a server with a different Process object than p.

In [ ]:
# Make sure we get a reasonable response.
# Should be a list of json objects with a title, a content and an url field.
!curl -X POST $HOST -d "q=baseball&n=1"

In [ ]:
!python -m parlai interactive --model-file zoo:blenderbot2/blenderbot2_400M/model --search_server $HOST